# 0 Setup

In [1]:
# Importing all required packages at the start of the notebook
import IPython

from qiime2 import Visualization

import qiime2 as q2
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
data_dir = "project_data" #Store the folder's path

# 1 Importing the data

In [3]:
%%bash -s $data_dir
mkdir -p "$1"

wget -nc --progress=dot:giga -P "$1" https://polybox.ethz.ch/index.php/s/uV06vmm96ZzB5eM/download/fungut_forward_reads.qza
wget -nc --progress=dot:giga -P "$1" https://polybox.ethz.ch/index.php/s/CA76kKFC9FApqpR/download/fungut_metadata.tsv

chmod -R +rxw "$1"

File ‘project_data/fungut_forward_reads.qza’ already there; not retrieving.

File ‘project_data/fungut_metadata.tsv’ already there; not retrieving.



In [9]:
# To check that our files are in the right place:
qza_file = f"{data_dir}/fungut_forward_reads.qza" #Store the sequences file
tsv_file = f"{data_dir}/fungut_metadata.tsv" #Store the sample metadata file
print("File exists?", os.path.exists(qza_file), os.path.exists(tsv_file))

File exists? True True


# 2 Feature table construction

## 2.1 First overview of our sample and quality score assessment 

In [4]:
# Visual summary of the data
! qiime demux summarize \
    --i-data $data_dir/fungut_forward_reads.qza \
    --o-visualization $data_dir/fungut_forward_reads_demux_seqs.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: project_data/fungut_forward_reads_demux_seqs.qzv


In [5]:
Visualization.load(f"{data_dir}/fungut_forward_reads_demux_seqs.qzv")

<visualization: Visualization uuid: 982abfbf-4d47-45b7-8012-20c466d46987>

## 2.2 Denoizing and creation of ASVs

In [6]:
! qiime dada2 denoise-single \
    --i-demultiplexed-seqs $data_dir/fungut_forward_reads.qza \
    --p-trunc-len 135 \
    --p-n-threads 3 \
    --p-max-ee 4 \
    --p-min-fold-parent-over-abundance 4 \
    --o-table $data_dir/dada2_table.qza \
    --o-representative-sequences $data_dir/dada2_rep_set.qza \
    --o-denoising-stats $data_dir/dada2_stats.qza

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved FeatureTable[Frequency] to: project_data/dada2_table.qza
Saved FeatureData[Sequence] to: project_data/dada2_rep_set.qza
Saved SampleData[DADA2Stats] to: project_data/dada2_stats.qza


In [7]:
! qiime metadata tabulate \
  --m-input-file $data_dir/dada2_stats.qza \
  --o-visualization $data_dir/dada2_stats.qzv

! qiime feature-table tabulate-seqs \
  --i-data $data_dir/dada2_rep_set.qza \
  --o-visualization $data_dir/dada2_rep_set.qzv

! qiime feature-table summarize \
  --i-table $data_dir/dada2_table.qza \
  --m-sample-metadata-file $data_dir/fungut_metadata.tsv \
  --o-visualization $data_dir/dada2_table.qzv

/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: project_data/dada2_stats.qzv
/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
Saved Visualization to: project_data/dada2_rep_set.qzv
/opt/conda/lib/python3.10/site-packages/unifrac/__init__.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 

In [8]:
Visualization.load(f"{data_dir}/dada2_stats.qzv")

<visualization: Visualization uuid: 6b7e26c8-954b-4d0f-be51-e6c81fd9bbf9>

In [9]:
Visualization.load(f"{data_dir}/dada2_rep_set.qzv")

<visualization: Visualization uuid: 9d0d13de-5372-4acc-8003-73d13fbd7168>

In [10]:
Visualization.load(f"{data_dir}/dada2_table.qzv")

<visualization: Visualization uuid: ebab64f3-14c3-480c-b390-68b3ae783731>